# RO47019: Intelligent Control Systems Practical Assignment
* Period: 2022-2023, Q3
* Course homepage: https://brightspace.tudelft.nl/d2l/home/500969
* Instructor: Cosimo Della Santina (C.DellaSantina@tudelft.nl)
* Teaching assistant: Ruben Martin Rodriguez (R.MartinRodriguez@student.tudelft.nl)
* (c) TU Delft, 2023

Make sure you fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`. Remove `raise NotImplementedError()` afterwards. Moreover, if you see an empty cell, please DO NOT delete it, instead run that cell as you would run all other cells. Please fill in your name(s) and other required details below:

In [ ]:
# Please fill in your names, student numbers, netID, and emails below.
STUDENT_1_NAME = ""
STUDENT_1_STUDENT_NUMBER = ""
STUDENT_1_NETID = ""
STUDENT_1_EMAIL = ""

In [ ]:
# Note: this block is a check that you have filled in the above information.
# It will throw an AssertionError until all fields are filled
assert STUDENT_1_NAME != ""
assert STUDENT_1_STUDENT_NUMBER != ""
assert STUDENT_1_NETID != ""
assert STUDENT_1_EMAIL != ""

### General announcements

* Do *not* share your solutions, and do *not* copy solutions from others. By submitting your solutions, you claim that you alone are responsible for this code.

* Do *not* email questions directly, since we want to provide everybody with the same information and avoid repeating the same answers. Instead, please post your questions regarding this assignment in the correct support forum on Brightspace, this way everybody can benefit from the response. If you do have a particular question that you want to ask directly, please use the scheduled Q&A hours to ask the TA.

* There is a strict deadline for each assignment. Students are responsible to ensure that they have uploaded their work in time. So, please double check that your upload succeeded to the Brightspace and avoid any late penalties.

* This [Jupyter notebook](https://jupyter.org/) uses `nbgrader` to help us with automated tests. `nbgrader` will make various cells in this notebook "uneditable" or "unremovable" and gives them a special id in the cell metadata. This way, when we run our checks, the system will check the existence of the cell ids and verify the number of points and which checks must be run. While there are ways that you can edit the metadata and work around the restrictions to delete or modify these special cells, you should not do that since then our nbgrader backend will not be able to parse your notebook and give you points for the assignment. You are free to add additional cells, but if you find a cell that you cannot modify or remove, please know that this is on purpose.

* This notebook will have in various places a line that throws a `NotImplementedError` exception. These are locations where the assignment requires you to adapt the code! These lines are just there as a reminder for youthat you have not yet adapted that particular piece of code, especially when you execute all the cells. Once your solution code replaced these lines, it should accordingly *not* throw any exceptions anymore.

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

# Problem 1 - Vision-based angle prediction (42.5p)

**Authors:** Tomás Coleman (T.Coleman@tudelft.nl), Chuhan Zhang (C.Zhang-8@tudelft.nl)


The following cells import all the necessary packages and external functions to properly run the code. Additionally, different dataset classes are created from the information gathered in the notebook 1a. Finally, different Pytorch data loaders will be created for each network architecture which will be introduced throughout the notebook where corresponds

In [ ]:
# Reloads the python files outside of this notebook automatically
%load_ext autoreload
%autoreload 2

# import all Python modules
from distutils.util import strtobool
from jax.config import config as jax_config

jax_config.update("jax_platform_name", "cpu")  # set default device to 'cpu'
jax_config.update("jax_enable_x64", True)  # double precision
from jax import random
from jax import numpy as jnp
import numpy as onp
import matplotlib.pyplot as plt
import os
from pathlib import Path

import torch
from torch.utils.data import Dataset, DataLoader
import snntorch as snn
from snntorch import surrogate
import torch.nn as nn
import torch.nn.functional as F
import os
from utils import *


# define boolean to check if the notebook is run for the purposes of autograding
AUTOGRADING = strtobool(os.environ.get("AUTOGRADING", "false"))
""
# create directory for datasets
datasets_dir = Path("datasets")
datasets_dir.mkdir(parents=True, exist_ok=True)

# create directory for plots
outputs_dir = Path("outputs")
outputs_dir.mkdir(parents=True, exist_ok=True)

# create directory for state dictionaries of neural network
statedicts_dir = Path("statedicts")
statedicts_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
class ThetaDataset(Dataset):
    def __init__(self, dataset):
        self.x = torch.tensor(dataset["th_pix_curr"], dtype=torch.float32) / 255.0
        self.y = torch.tensor(dataset["th_curr_ss"], dtype=torch.float32)

    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]

        return x, y

    def __len__(self):
        num_samples = self.x.shape[0]

        return num_samples


class TrigDataset(Dataset):
    def __init__(self, dataset):
        self.x = torch.tensor(dataset["th_pix_curr"], dtype=torch.float32) / 255.0
        y_cos = torch.cos(torch.tensor(dataset["th_curr_ss"], dtype=torch.float32))
        y_sin = torch.sin(torch.tensor(dataset["th_curr_ss"], dtype=torch.float32))
        self.y = torch.cat((y_sin, y_cos), dim=-1)

    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]

        return x, y

    def __len__(self):
        num_samples = self.x.shape[0]

        return num_samples


class CNNDataset(Dataset):
    def __init__(self, dataset):
        self.x = (
            torch.tensor(
                onp.transpose(dataset["th_pix_curr"], (0, 3, 1, 2)), dtype=torch.float32
            )
            / 255.0
        )
        y_cos = torch.cos(torch.tensor(dataset["th_curr_ss"], dtype=torch.float32))
        y_sin = torch.sin(torch.tensor(dataset["th_curr_ss"], dtype=torch.float32))
        self.y = torch.cat((y_sin, y_cos), dim=-1)

    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        return x, y

    def __len__(self):
        num_samples = self.x.shape[0]

        return num_samples


class SNNDataset(Dataset):
    def __init__(self, path, num_itr, num_data):
        self.path = path
        self.num_itr = num_itr
        self.num_data = num_data

    def __len__(self):
        return int(self.num_itr * self.num_data)

    def __getitem__(self, index):
        if index < int(self.num_data * self.num_itr):
            spike_path = "spike" + str(index) + ".pt"
            label_path = "target" + str(index) + ".pt"
            spike_out = torch.load(os.path.join(self.path, spike_path))
            label_out = torch.load(os.path.join(self.path, label_path))
            return spike_out, label_out
        else:
            raise IndexError

In [ ]:
def load_training_dataset_dataloader(
    _filepath, _rng: random.KeyArray, _val_ratio=0.3, batch_size=64, model_type="theta"
):
    assert 0.0 <= _val_ratio <= 1.0, "Validation ratio needs to be in interval [0, 1]."

    _dataset = jnp.load(_filepath)
    num_samples = _dataset["th_curr_ss"].shape[0]

    indices = jnp.arange(num_samples)
    shuffled_indices = random.permutation(_rng, indices)
    num_train_samples = int((1 - _val_ratio) * num_samples)
    split_config = jnp.array(
        [
            num_train_samples,
        ]
    )
    train_indices, val_indices = jnp.split(shuffled_indices, split_config)

    _train_ds, _val_ds = {}, {}
    for key, val in _dataset.items():
        _train_ds[key] = val[train_indices]
        _val_ds[key] = val[val_indices]

    if model_type == "theta":
        train_data = ThetaDataset(_train_ds)
        val_data = ThetaDataset(_val_ds)
    elif model_type == "trig":
        train_data = TrigDataset(_dataset)
        val_data = TrigDataset(_val_ds)
    elif model_type == "cnn":
        train_data = CNNDataset(_dataset)
        val_data = CNNDataset(_val_ds)

    _train_dataloader = torch.utils.data.DataLoader(
        train_data, batch_size=batch_size, shuffle=True
    )
    _val_dataloader = torch.utils.data.DataLoader(
        val_data, batch_size=batch_size, shuffle=True
    )

    return _train_dataloader, _val_dataloader


def load_test_dataset_dataloader(
    _filepath, _rng: random.KeyArray, batch_size=64, model_type="theta"
):
    _dataset = jnp.load(_filepath)
    num_samples = _dataset["th_curr_ss"].shape[0]

    if model_type == "theta":
        print("Theta Dataset")
        _data = ThetaDataset(_dataset)

    elif model_type == "trig":
        print("Trig Dataset")
        _data = TrigDataset(_dataset)
    elif model_type == "cnn":
        print("CNN dataset")
        _data = CNNDataset(_dataset)

    _dataloader = torch.utils.data.DataLoader(
        _data, batch_size=batch_size, shuffle=False
    )
    return _dataloader, _dataset

One last function, evaluate_model, is also defined to evaluate the outcomes of the different models in their respective datasets and display a heatmap of the prediction errors. You do not have to worry about its workings nor interface since you are not required to use it yourself

In [ ]:
def evaluate_model(
    _model, test_dataset, model="theta", file="nothing.pdf", num_bins: int = 10
):
    filepath = str(outputs_dir / file)
    th = test_dataset["th_curr_ss"]

    # The pixel values need to be divided by 255.0 to match the input of the dataset classes
    if model == "cnn":
        x = onp.transpose(
            onp.array(test_dataset["th_pix_curr"] / 255.0, dtype=onp.float32),
            (0, 3, 1, 2),
        )
        x = torch.tensor(x)
        x_len = x.size()[0]
        batch_size = 100
        batch_num = int(int(x_len) / int(batch_size))
        y_hat = torch.zeros((0, 4))
        for i in range(batch_num):
            x_batch = x[int(i * batch_size) : int((i + 1) * batch_size)]
            y_batch = _model(x_batch).detach()
            y_hat = torch.cat((y_hat, y_batch), dim=0)
        y_hat = y_hat.numpy()
    else:
        y_hat = (
            _model(
                torch.tensor(
                    onp.array(test_dataset["th_pix_curr"] / 255.0, dtype=onp.float32)
                )
            )
            .detach()
            .numpy()
        )  # need to convert back to np array

    if model == "theta":
        y = th
    else:
        y_cos = onp.cos(th)
        y_sin = onp.sin(th)

        y = onp.concatenate((y_sin, y_cos), axis=-1)

        y = trig_to_theta_numpy(y)
        y_hat = trig_to_theta_numpy(y_hat)

    error = y_hat - y
    error = onp.abs(error)

    extent = onp.array(
        [
            [-onp.pi, onp.pi],
            [-onp.pi, onp.pi],
        ]
    )
    heatmap, xedges, yedges = onp.histogram2d(
        x=th[:, 0], y=th[:, 1], bins=num_bins, range=extent
    )

    avg_bins = onp.zeros((num_bins, num_bins))
    for i in range(avg_bins.shape[0]):
        if i == 0:
            # also include the left edge for the first bin
            xcond = (xedges[i] <= th[:, 0]) & (th[:, 0] <= xedges[i + 1])
        else:
            xcond = (xedges[i] < th[:, 0]) & (th[:, 0] <= xedges[i + 1])

        for j in range(avg_bins.shape[1]):
            if j == 0:
                # also include the left edge for the first bin
                ycond = (yedges[j] <= th[:, 1]) & (th[:, 1] <= yedges[j + 1])
            else:
                ycond = (yedges[j] < th[:, 1]) & (th[:, 1] <= yedges[j + 1])

            # combine condition for th1 and th2
            bin_cond = xcond & ycond

            # save the average error for each bin
            avg_bins[i, j] = onp.mean(error[bin_cond, :])

    print("Mean prediction error across bins:", onp.mean(avg_bins))

    plt.clf()

    plt.imshow(avg_bins, extent=extent.flatten())
    plt.title("Heatmap of prediction error")
    plt.xlabel("link 1 angles (rad)")
    plt.ylabel("link 2 angles (rad)")

    plt.colorbar()
    plt.savefig(filepath)
    plt.show()


def trig_to_theta(trig_data):
    trig_len = trig_data.shape[0]
    theta = torch.zeros(trig_len, 2)
    for i in range(trig_len):
        theta[i, 0] = torch.atan2(trig_data[i, 0], trig_data[i, 2])
        theta[i, 1] = torch.atan2(trig_data[i, 1], trig_data[i, 3])

    return theta


# Create a conversion function that supports numpy arrays instead of torch tensors
def trig_to_theta_numpy(trig_data):
    trig_len = trig_data.shape[0]
    theta = onp.zeros((trig_len, 2))
    for i in range(trig_len):
        theta[i, 0] = onp.arctan2(trig_data[i, 0], trig_data[i, 2])
        theta[i, 1] = onp.arctan2(trig_data[i, 1], trig_data[i, 3])

    return theta

In [ ]:
# set seed for jax
rng = random.PRNGKey(seed=42)
rng, init_rng = random.split(rng)

## Task 1.1: Learn to predict angles (7.5p)

We are going to create models that try to predict the link angles $\hat{\theta}$ given an image of the robot, so $\theta \approx \hat{\theta} =M(x)$.

### 1.1 - Prepare data 
Run the functions below to load the data generated in `task_1a_generate_data.ipynb`, split the training into the 70/30 train/validation split, set the batch size to and put them in a dataloader. The data is first put in the class `ThetaDataset`. This class converts the `uint8` pixel values from a range of 0-255 to float values in the range `0-1`. The class also provides methods for the dataloader to retreive the pixel obervations `.x` and test labels `y`. The dataloader is set to Shuffle the data randomly before enumerating during the training loop.

In [ ]:
if not AUTOGRADING:
    train_loader_theta, val_loader_theta = load_training_dataset_dataloader(
        str(datasets_dir / "dataset_double_pendulum_train.npz"),
        _rng=init_rng,
        model_type="theta",
    )
    test_loader_theta, _ = load_test_dataset_dataloader(
        str(datasets_dir / "dataset_double_pendulum_test.npz"),
        _rng=init_rng,
        model_type="theta",
    )

### 1.1 - Create the model (1.5p)
Here, create a PyTorch model class called NeuralNetworkTheta()
- Start by flattening the input with `torch.nn.Flatten()`.
- Then, add a fully connected hidden layer `torch.nn.Linear(...)` of 128 units and ReLU activation.
- Finally, add a final fully connected linear layer `torch.nn.Linear(...)`without activation. The number of units must match the dimension of our target data, which is 2, as we try to predict the angles $\theta_1$ and $\theta_2$.


In [ ]:
"""
NN Model with Theta as the output
"""

""" TASK 1.1: CREATE MODEL HERE """


class NeuralNetworkTheta(nn.Module):
    def __init__(self):
        super().__init__()
        # YOUR CODE HERE
        raise NotImplementedError()
"""TASK1.1: END"""
model_theta = NeuralNetworkTheta()

total_params_theta = sum(p.numel() for p in model_theta.parameters())
print("total number of model parameters: ", total_params_theta)

### 1.1 - Train the model (2p) 

Now that the model is defined, we can train it using the training dataset. We have a train-validation split of 0.3 and a batch size of 64 that is shuffled every epoch. Train the model for 100 epochs with the training data `train_loader_theta`. Tune the learning rate in the optimizer to give the best performance on the validation and test set over the 100 epochs. Do this 10 times and record the prediction error of the final model of each of the 10 runs on the test dataset `test_loader_theta` using the function `evaluate_model_test_data_theta`. For each training loop, it would be useful to test the model performance on the valedation set to see if it is overfitting on the training data or whether it can generalize well. **hint:** 
- reduce the number of epochs and lower the number of runs to `1` while getting your model working and setting the optimal learning rate. 
- The try variying the the learning rate in the range of `1e-1` to `1e-5`.
 

In [ ]:
# Training parameters
optimizer = torch.optim.SGD(model_theta.parameters(), lr=1e-3)
# We choose the mean square loss as it is a regression problem
loss_fn = torch.nn.MSELoss()
error_fn = torch.nn.L1Loss()
num_epochs = 100

# path where we will save the trained model
model_theta_path = statedicts_dir / "task_1-1_model_theta.pt"


def evaluate_model_test_data_theta(_model, _test_loader):
    running_loss_test = 0.0
    running_error_test = 0.0
    count = 0
    with torch.no_grad():
        for i, data_test in enumerate(_test_loader):
            count += 1
            inputs_test, labels_test = data_test
            batch = labels_test.shape[0]
            est_th = _model(inputs_test)
            loss_test = loss_fn(est_th, labels_test)
            error_test = error_fn(est_th, labels_test)
            running_loss_test += loss_test.item()
            running_error_test += error_test.item()
    print(
        f"Loss on test data: {running_loss_test / count:.3f}, Prediction error of model on test data: {running_error_test / count:.3f}"
    )
    return running_error_test / count

In [ ]:
if not AUTOGRADING:
    """TASK 1.1: TRAIN MODEL HERE"""

    num_runs = 10  # Change to 1 until you get it to work once
    pred_error_theta = onp.zeros((num_runs))

    for run in range(num_runs):
        # this code reinitializes the parameters of the model on each loop
        model_theta = NeuralNetworkTheta()
        optimizer = torch.optim.SGD(model_theta.parameters(), lr=1e-3)

        # YOUR CODE HERE
        raise NotImplementedError()
        print(f"run {run+1} finished")
        pred_error_theta[run] = evaluate_model_test_data_theta(
            model_theta, test_loader_theta
        )
    print("average prediction error: ", onp.mean(pred_error_theta))
    """TASK 1.1: END"""

In [ ]:
# If working on CodeSpaces, run this cell before running the next model to free up memory
if not AUTOGRADING:
    # save the jitted model of the last run
    model_theta_scripted = torch.jit.script(model_theta)
    model_theta_scripted.save(str(model_theta_path))

    %xdel train_loader_theta
    %xdel val_loader_theta
    %xdel test_loader_theta
    %xdel model_theta
    %xdel model_theta_scripted

In [ ]:
if not AUTOGRADING:
    # load jitted model of the last run
    model_theta_scripted = torch.jit.load(str(model_theta_path))

    _, test_dataset_theta = load_test_dataset_dataloader(
        str(datasets_dir / "dataset_double_pendulum_test.npz"),
        _rng=init_rng,
        model_type="theta",
    )

    evaluate_model(
        model_theta_scripted,
        test_dataset_theta,
        model="theta",
        file="task_1-1_model_theta_prediction_error.pdf",
    )

    %xdel model_theta_scripted
    %xdel test_dataset_theta

### 1.1 - Analyse Model performance (4p)
Analyze the prediction accuracy of the model on the test data from heat map of the loss outputted from the `evaluate_model` function. Where does this model have the lowest accuracy? What could be an explanation for the loss of accuracy in those regions? **2p** (Answer in the cell below)

YOUR ANSWER HERE

In general, a separate test dataset is used to evaluate a trained model. Why? **(2p)** (Answer in the cell below)

YOUR ANSWER HERE

## Task 1.2: Indirectly predict the angle
We are going to improve the accuracy by pre-processing the target data. Specifically, we will create a model $M_{trig}$ that learns to predict $\sin(\theta)$ and $\cos(\theta)$ for both links instead of directly predicting $\theta$. Then, we can use the trigonometric relation to retrieve an estimate of $\theta$ for both links.

Note: In practice you would use the `atan2` implementation, as the regular arctangent only covers $[-\frac{1}{2}\pi, \frac{1}{2}\pi]$} $\theta=\arctan(\frac{\sin(\theta)}{\cos(\theta)})$ 

### 1.2 Prepare the data (0p)

Run the functions below to load the data generated in `task_1a_generate_data.ipynb`, split the training into the 70/30 train/validation split, set the batch size to and put them in a dataloader. The data is first put in the class `TrigDataset`. This class again converts the `uint8` pixel values from a range of 0-255 to float values in the range `0-1` while also calculating the values of $\mathrm{sin}(\theta_1), \mathrm{sin}(\theta_2), \mathrm{cos}(\theta_1), \mathrm{cos}(\theta_2)$ to be used as labels instead of just $\theta_1, \theta_2$ as was done in the `ThetaDataset` class. The dataloader is again set to shuffle the data randomly before enumerating during the training loop.

In [ ]:
if not AUTOGRADING:
    train_loader_trig, val_loader_trig = load_training_dataset_dataloader(
        datasets_dir / "dataset_double_pendulum_train.npz",
        _rng=init_rng,
        model_type="trig",
    )
    test_loader_trig, _ = load_test_dataset_dataloader(
        datasets_dir / "dataset_double_pendulum_test.npz",
        _rng=init_rng,
        model_type="trig",
    )

### 1.2 - Create the model (1.5p)

Copy the model you created in Task 1.1. Change the number of hidden units of the final layer from 2 to 4. We do so, because we now want to predict two outputs ($\sin(\theta)$, $\cos(\theta)$) for both link angles, instead of only two outputs, $\theta_1$ and $\theta_2$.

In [ ]:
""" TASK 1.2: CREATE MODEL HERE """


class NeuralNetworkTrig(nn.Module):
    def __init__(self):
        super().__init__()
        # YOUR CODE HERE
        raise NotImplementedError()
"""TASK1.2: END"""

model_trig = NeuralNetworkTrig()
total_params_trig = sum(p.numel() for p in model_trig.parameters())
print("Total number of neural network parameters: ", total_params_trig)

### 1.2 - Train the model (2p)

Now train a model with the same training parameters defined below as in task 1.1 but using the training data `train_loader_trig`. Tune the learning rate in the optimizer to give the best performance on the validation and test set over the 100 epochs. Do this 10 times and record the prediction error of the final model of each of the 10 runs on the test dataset `test_loader_trig` using the function `evaluate_model_test_data_trig`. **Hint:** 
- reduce the number of epochs and lower the number of runs to `1` while getting your model working. 
- The try in the range `1e-1` to `1e-5` for the learning rate

In [ ]:
# Training parameters
optimizer = torch.optim.SGD(model_trig.parameters(), lr=1e-3)
loss_fn = torch.nn.MSELoss()
error_fn = torch.nn.L1Loss()
num_epochs = 100

# path where we will save the trained model
model_trig_path = statedicts_dir / "task_1-2_model_trig.pt"


def evaluate_model_test_data_trig(_model, _test_loader):
    running_loss_test = 0.0
    running_error_test = 0.0
    count = 0
    with torch.no_grad():
        for i, data_test in enumerate(_test_loader):
            count += 1
            inputs_test, labels_test = data_test
            batch = labels_test.shape[0]
            est = _model(inputs_test)
            est_th = trig_to_theta(est)
            label_th = trig_to_theta(labels_test)
            loss_test = loss_fn(est_th, label_th)
            error_test = error_fn(est_th, label_th)
            running_loss_test += loss_test.item()
            running_error_test += error_test.item()
    print(
        f"Loss on test data: {running_loss_test / count:.3f}, Prediction error of model on test data: {running_error_test / count:.3f}"
    )

    return running_error_test / count

In [ ]:
if not AUTOGRADING:
    """TASK 1.2: TRAIN MODEL HERE"""
    num_runs = 10
    pred_error_trig = onp.zeros((num_runs))

    for run in range(num_runs):
        model_trig = NeuralNetworkTrig()
        optimizer = torch.optim.SGD(model_trig.parameters(), lr=1e-3)
        # YOUR CODE HERE
        raise NotImplementedError()
        print(f"run {run+1} finished")
        pred_error_trig[run] = evaluate_model_test_data_trig(
            model_trig, test_loader_trig
        )
    print("average prediction error: ", onp.mean(pred_error_trig))
    """TASK 1.2: END"""

In [ ]:
# If working on CodeSpaces, run this cell before running the next model to free up memory
if not AUTOGRADING:
    # save the jitted model of the last run
    model_trig_scripted = torch.jit.script(model_trig)
    model_trig_scripted.save(str(model_trig_path))

    %xdel train_loader_trig
    %xdel val_loader_trig
    %xdel test_loader_trig
    %xdel model_trig
    %xdel model_trig_scripted

In [ ]:
if not AUTOGRADING:
    # load jitted model of the last run
    model_trig_scripted = torch.jit.load(str(model_trig_path))

    _, test_dataset_trig = load_test_dataset_dataloader(
        datasets_dir / "dataset_double_pendulum_test.npz",
        _rng=init_rng,
        model_type="trig",
    )

    evaluate_model(
        model_trig_scripted,
        test_dataset_trig,
        model="trig",
        file="task_1-2_model_trig_prediction_error.pdf",
    )

    %xdel test_dataset_trig
    %xdel model_trig_scripted

### 1.2 - Analyse Model performance (4p)
Compare the prediction estimates for $M_{trig}$ with the plot for $M_θ$. Why does indirectly predicting the angle improve the prediction accuracy? **(2p)** (Answer in the cell below)

YOUR ANSWER HERE

Why is it not sufficient to predict only sin(θ) and use its inverse θ = arcsin(sin(θ)) to get an estimate of the angle? **(2p)**

YOUR ANSWER HERE

## Task 1.3: Indirectly predict the angles with a Convolutional Neural Network (10)
Instead of using a vanilla fully-connected neural network, we will build a prediction model $M_{cnn}$ that uses a convolutional neural network (CNN). 

### 1.3 - Prepare the data

Run the functions below to load the data generated in `task_1a_generate_data.ipynb`, split the training into the 70/30 train/validation split, set the batch size to and put them in a dataloader. The data is first put in the class `CNNDataset`. This class again converts the `uint8` pixel values from a range of 0-255 to float values in the range `0-1` while also calculating the values of $\mathrm{sin}(\theta_1), \mathrm{sin}(\theta_2), \mathrm{cos}(\theta_1), \mathrm{cos}(\theta_2)$ to be used as labels instead of just $\theta_1, \theta_2$ as was done in the `ThetaDataset` class. We also permutate the order of the pixel ovbservations from `(number of samples, pixel rows, pixel columns, color channels)` to `(number of samples, color channels, pixel rows, pixel columns, )` as this is the order that the 2d convolution layer expects. The dataloader is again set to shuffle the data randomly before enumerating during the training loop.


In [ ]:
if not AUTOGRADING:
    train_loader_cnn, val_loader_cnn = load_training_dataset_dataloader(
        datasets_dir / "dataset_double_pendulum_train.npz",
        _rng=init_rng,
        model_type="cnn",
    )
    test_loader_cnn, _ = load_test_dataset_dataloader(
        datasets_dir / "dataset_double_pendulum_test.npz",
        _rng=init_rng,
        model_type="cnn",
    )

### 1.3 - Create the model (2p)
Again, create sequential PyTorch model class and define the following architecture with Convolutional and pooling layers. 
- Start with a CNN layer `torch.nn.Conv2d(...)` with 32 filters, kernel size of `3x3` and a ReLU activation function.
- Then add a max pooling layer `torch.nn.MaxPool2d(...)` with a pool size of `2x2`.
- Add another convolutional layer with 10 filters and a ReLU activation function.
- Then add another max pooling layer `torch.nn.MaxPool2d(...)` with a pool size of `2x2`.
- Flatten the output of the pooling layer
- Add a fully connected layer of 30 with a relu activation function.
- Finally, add a fully connected layer without activation. Remember, the number of ouput units must match the dimension of the target data, which is 4 as we will predict the trigonometric function again. 

**note:* No zero padding is needed as the images have enough whitespace at the edges.

In [ ]:
"""TASK 1.3: CREATE MODEL HERE"""


# YOUR CODE HERE
raise NotImplementedError()
"""TASK 1.3: END HERE"""

model_cnn = NeuralNetworkCNN()
total_params_cnn = sum(p.numel() for p in model_cnn.parameters())
print("total number of model parameters: ", total_params_cnn)

### 1.3 - Train the model (2p)

Train the model with the same training parameters defined below as in task 1.1 and 1.2 but using the training data `train_loader_cnn`. Again, Tune the learning rate in the optimizer to give the best performance on the validation and test set over the 100 epochs. Do this 10 times and record the prediction error of the final model of each of the 10 runs on the test dataset `test_loader_cnn` using the function `evaluate_model_test_data_cnn`. Don't forget to reinitialize the parameters on each run! **Hints:** 
- reduce the number of epochs and lower the number of runs to `1` while getting your model working. 
- The try in the range of 1e-1 to 1e-5 for the learning rate

#### One last thing!
A Convolutional neural network that can predict angles is required for task 2b. Therefore, focus getting the perfect learning and optimal model accuracy with this Neural Network to see how far you can increase the prediction accuracy of link angles. Be warned, the accuracy of your model will affect the performance of you controller in task 2b.

In [ ]:
# Training parameters
optimizer = torch.optim.SGD(model_cnn.parameters(), lr=1e-3)
loss_fn = torch.nn.MSELoss()
error_fn = torch.nn.L1Loss()
num_epochs = 100

# path where we will save the trained model
model_cnn_path = statedicts_dir / "task_1-3_model_cnn.pt"


def evaluate_model_test_data_cnn(_model, _test_loader):
    """
    evaluates the model_cnn on the
    """
    running_loss_test = 0.0
    running_error_test = 0.0
    count = 0
    with torch.no_grad():
        for i, data_test in enumerate(_test_loader):
            count += 1
            inputs_test, labels_test = data_test
            batch = labels_test.shape[0]
            est = _model(inputs_test)
            est_th = trig_to_theta(est)
            label_th = trig_to_theta(labels_test)
            loss_test = loss_fn(est_th, label_th)
            error_test = error_fn(est_th, label_th)
            running_loss_test += loss_test.item()
            running_error_test += error_test.item()

    print(
        f"Loss on test data: {running_loss_test / count:.3f}, Prediction error of model on test data: {running_error_test / count:.3f}"
    )
    return running_error_test / count

In [ ]:
if not AUTOGRADING:
    """TASK 1.3: TRAIN MODEL HERE"""

    num_runs = 1
    pred_error_cnn = onp.zeros((num_runs))

    for run in range(num_runs):
        model_cnn = NeuralNetworkCNN()
        optimizer = torch.optim.SGD(model_cnn.parameters(), lr=1e-1)
        # YOUR CODE HERE
        raise NotImplementedError()

        print(f"run {run+1} finished")
        pred_error_cnn[run] = evaluate_model_test_data_cnn(model_cnn, test_loader_cnn)
    print("average prediction error: ", onp.mean(pred_error_cnn))

    print("Finished Training")

    """TASK 1.3: END"""

In [ ]:
# If working on CodeSpaces, run this cell before running the next model to free up memory
if not AUTOGRADING:
    # save the jitted model of the last run
    model_cnn_scripted = torch.jit.script(model_cnn)
    model_cnn_scripted.save(str(model_cnn_path))

    %xdel train_loader_cnn
    %xdel val_loader_cnn
    %xdel test_loader_cnn
    %xdel model_cnn
    %xdel model_cnn_scripted

In [ ]:
if not AUTOGRADING:
    # load jitted model of the last run
    model_cnn_scripted = torch.jit.load(str(model_cnn_path))

    _, test_dataset_cnn = load_test_dataset_dataloader(
        datasets_dir / "dataset_double_pendulum_test.npz",
        _rng=init_rng,
        model_type="cnn",
    )

    evaluate_model(
        model_cnn_scripted,
        test_dataset_cnn,
        model="cnn",
        file="task_1-3_model_cnn_prediction_error.pdf",
    )

    %xdel test_dataset_cnn
    %xdel model_cnn_scripted

### 1.3 - Analyse and Compare Model performance 
Make a comparison of the different models (i.e. M θ, M trig, M cnn) based on the average prediction accuracy
on the test dataset and the number of trainable parameters. Which model would you prefer and why? (answer in the cell below)

YOUR ANSWER HERE

If you change the activation of the last fully connected layer to ReLU, the prediction accuracy completely deteriorates. Why?

YOUR ANSWER HERE

## Task 1.4: Variance across runs (2.5p) 
In the previous tasks, the prediction accuracy varied across different runs even though the underlying code and dataset remained unchanged.

Use the code cell below to set the seed for PyTorch. Train the model for a seed of 0 and then for seed of 1, recording the average prediction accuracy and standard deviation for both.  

In [ ]:
# Important: only run this cell once as it resets your results for all seeds
# initialize dictionary to keep track of prediction error across seeds
pred_error_sds = {}

In [ ]:
# set seed to either 0 or 1
seed = 0

Copy the architecture of $M_{\theta}$ to create a new model below. Don't forget to comment on the code to set the seed again when working on the other tasks.

In [ ]:
""" TASK 1.4: CREATE MODEL HERE """


class NeuralNetworkThetaSeed(nn.Module):
    def __init__(self):
        super().__init__()
        # YOUR CODE HERE
        raise NotImplementedError()
"""TASK1.4: END"""
model_theta_seed = NeuralNetworkThetaSeed()

Set up a training loop as done for the task 1.3 and run the training 10 times while having the seed value as 0 and then run it again for a seed of 1. Use the same datasets as in task 1.3. **(0.5p)**. 

In [ ]:
if not AUTOGRADING:
    train_loader_theta, val_loader_theta = load_training_dataset_dataloader(
        str(datasets_dir / "dataset_double_pendulum_train.npz"),
        _rng=init_rng,
        model_type="theta",
    )
    test_loader_theta, _ = load_test_dataset_dataloader(
        str(datasets_dir / "dataset_double_pendulum_test.npz"),
        _rng=init_rng,
        model_type="theta",
    )

# Training parameters
optimizer = torch.optim.SGD(model_theta_seed.parameters(), lr=1e-3)
# We choose the mean square loss as it is a regression problem
loss_fn = torch.nn.MSELoss()
num_epochs = 30

# define the path to save the model to
model_theta_seed_path = str(statedicts_dir / f"task_1-4_model_theta_seed-{seed}.pt")

num_runs = 10
pred_error_sds[seed] = onp.zeros((num_runs,))

In [ ]:
if not AUTOGRADING:
    """TASK 1.4: TRAIN MODEL HERE"""
    for run in range(num_runs):
        torch.manual_seed(seed=seed)
        model_theta_seed = NeuralNetworkThetaSeed()
        optimizer = torch.optim.SGD(model_theta_seed.parameters(), lr=1e-3)
        # YOUR CODE HERE
        raise NotImplementedError()
    """TASK 1.4: END"""

In [ ]:
# If working on CodeSpaces, run this cell before running the next model to free up memory
if not AUTOGRADING:
    # jit the model of the last run
    model_theta_seed_scripted = torch.jit.script(model_theta_seed)

    # save the jitted model of the last run
    model_theta_seed_scripted.save(str(model_theta_seed_path))

    %xdel train_loader_theta
    %xdel val_loader_theta
    %xdel test_loader_theta
    %xdel model_theta_seed
    %xdel model_theta_seed_scripted

In [ ]:
if not AUTOGRADING:
    # load the jitted model of the last run of the currently selected seed
    model_theta_seed_scripted = torch.jit.load(str(model_theta_seed_path))

    _, test_dataset_theta = load_test_dataset_dataloader(
        str(datasets_dir / "dataset_double_pendulum_test.npz"),
        _rng=init_rng,
        model_type="theta",
    )

    evaluate_model(
        model_theta_seed_scripted,
        test_dataset_theta,
        model="theta",
        file=f"task_1-4_model_theta_prediction_error_seed-{seed}.pdf",
    )

    %xdel test_dataset_theta
    %xdel model_theta_seed_scripted

- What do you observe? (1p)

- What is the benefit of seeding the pseudo-random generator in practice? **(1p)**

Write your answers in the cell below

YOUR ANSWER HERE

## Task 1.5: Spiking neural networks (15p)

If our goal is to control the double pendulum, the angular velocity $\dot{\theta}$ is generally also required. However, temporal information cannot be extracted from individual images. We are going to utilize the temporal advantage of the spiking neural networks (SNN) to predict the angular velocity of each link. Instead of static individual images, we adopt event-based data as the input.

### 1.5 - Warm Up
Use the Jupyter notebook `task_1-5_SNN_warmup.ipynb` to understand the neuron structure of SNNs, construct a LIF neuron model and a simple fully connected SNN.

### 1.5 - Prepare the dataset

In this part, the dataloader has been defined, and the two data sets `train_loader_snn` and `test_loader_snn` have been generated, which can be directly used to train the network. The two datasets are the same size. The batch_size has chosen 16. And we shuffle the dataset by setting shuffle to true. At the same time, to make the size of the data set just divisible by batch_size, we set drop_last to true to discard the last set of data that is less than one batch. At the same time, we did not set the validation set.

Therefore, the size of each piece of data in dataloader is: （batch_size: 16, time_step: 20, channels: 2, size: 32, size: 32）.

In [ ]:
# This part has to be same as task_1a_generate_data.ipynb
train_th1_range = jnp.arange(-jnp.pi / 6.0, jnp.pi / 6.0, jnp.pi / 30.0)
train_th2_range = jnp.arange(-jnp.pi, jnp.pi, jnp.pi / 6.0)
test_th1_range = jnp.arange(-jnp.pi / 6.0, jnp.pi / 6.0, jnp.pi / 30.0)
test_th2_range = jnp.arange(-jnp.pi, jnp.pi, jnp.pi / 6.0)

TRAIN_NUM_DATA = len(train_th1_range) * len(train_th2_range)
TEST_NUM_DATA = len(test_th1_range) * len(test_th2_range)
NUM_SNN_DATA = 5

In [ ]:
if not AUTOGRADING:
    train_set_snn = SNNDataset(
        datasets_dir / "event_based_data" / "train",
        TRAIN_NUM_DATA,
        NUM_SNN_DATA,
    )
    train_loader_snn = DataLoader(
        train_set_snn, batch_size=16, shuffle=True, drop_last=True
    )

    test_set_snn = SNNDataset(
        datasets_dir / "event_based_data" / "test",
        TEST_NUM_DATA,
        NUM_SNN_DATA,
    )
    test_loader_snn = DataLoader(
        test_set_snn, batch_size=16, shuffle=True, drop_last=True
    )

### 1.5 - Create the model (4p)

We create a class with the name `SnnModel` extending `nn.Module`. Please complete the implementation below. In thi case, we adopt the LIF model for neurons.

The structure of the network is as follows:

* Start with a convolutional layer with $16$ output channels and a kernel size of `5x5`.
* Then, add a max pooling layer with a pooling size of `4x4`.
* Add a lif operation to convert the input current from the convolution into output spikes.
* Add the second convolutional layer with $16$ as output channel size, kernel size of `3x3`, then add a lif operation.
* Add the third convolutional layer with $16$ as output channel size, kernel size of `3x3`, then add a lif operation.
* All above parts are in a recurrent way. Then, the SNN part is finished. All last layer's membrane potential in each time step will be stacked together to get the SNN result.
* This final step is to process the SNN's results as usual. Flatten the obtained membrane potential and feed it into two fully connected layers with the ReLU activation fuction in between. We recommend the input size of the 2nd fully connected layer to be $640$. The final output number must match the dimension of our target data which is 2.

Meanwhile for LIF model, we use a `fast_sigmoid` function as the surrogate function, and set the decay rate `beta` to $0.8$. It is worth noting that we need to use a recurrent approach to process event-based data. Therefore,

* we should use the for-loop to traverse each time step of the input event-based data. For the data of each time step, we can use the convolution operation to process the data as usual. Each convolution operation is followed by a layer of SNN neurons. Finally, the membrane potential of the last layer of all time steps is used as the output. Therefore, the expected output format of the SNN part should be `[batch_size: 16, time_step: 20, conv_channel: (should be the channel size of the last convolutional layer), size: (output size), size: (output size)]`.
* The first dimension of the input data is batch size, but not the time steps. So we need to transpose the input tensor first and then use for-loop to process it.
* The final non-linear layer (two fully connected layers with ReLU) should be the conventional neurons instead of SNN neurons.
* Do not forget to initialize the LIF neurons!

**Note**: What's the differences between the fully connected (FC) layer here compared to the fully connected layer in the SNN structure in `task_1-5_SNN_warmup.ipyn`?

The FC layer in fully-connected SNN structure is used to build a SNN model to process the input spikes. So it should be implemented in the for-loop, and followed by a lif operation (LIF neuron models). However, the FC layer in this assignment is used to map the last  membrane potential steps to the final output. So it should be implemented outside of the for-loop and in the conventional way (i.e. like in the conventional neuron models).

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()
"""TASK1.5: END"""

model_snn = SnnModel()

### 1.5 - Train the model (3p)

Train the SNN model in the same way with other models. We recommend using `1e-3` as the learning rate and training the model for 200 epochs.

In [ ]:
optimizer = torch.optim.Adam(model_snn.parameters(), lr=1e-3)
loss_fn = torch.nn.MSELoss()
num_epochs = 200

In [ ]:
onp.random.seed(42)
torch.manual_seed(42)
if not AUTOGRADING:
    """TASK 1.5: TRAIN MODEL HERE"""
    # YOUR CODE HERE
    raise NotImplementedError()
    """TASK 1.5: END"""

In [ ]:
# Evaluate the SNN model with test data
if not AUTOGRADING:
    torch.save(model_snn.state_dict(), str(statedicts_dir / "task_1-5_model_snn.pt"))

    with torch.no_grad():
        model_snn = model_snn.eval()
        print_loss = 0.0
        count = 0
        for i, (events, targets) in enumerate(test_loader_snn):
            output = model_snn(events)
            loss = loss_fn(output, targets)
            print_loss += loss.item()
            count = count + 1

        print_loss = print_loss / count
        print(f"The MSE loss of the model on the test dataset is: {print_loss}")

### 1.5 - Analysis of results (8p)

Does your loss decrease step by step? During the training process, is the training speed faster or slower comparing to CNN and why? Please analyse the advantages and disadvantages of SNN. **(8p)**

YOUR ANSWER HERE